In [20]:
import pandas as pd
import pickle
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer


In [21]:
with open('finalized_model.sav', 'rb') as fid:
    model = pickle.load(fid)

In [28]:
courses_df = pd.read_csv("clean_phone_data.csv")
dt = {
    # string
    'colors': 'string',
    'build_material': 'string',
    'has_wifi': 'string',
    'has_bluetooth': 'string',
    'has_cardslot': 'string',
    'camera_list_resolution_back': 'string',
    'camera_list_resolution_front': 'string',
    'camera_resolution_back_recording': 'string',
    'camera_resolution_front_recording': 'string',
    'os': 'string',
    'storage_type': 'string',
    'brand': 'string',

    # float
    'battery_size': 'float',
    'battery_endurance': 'float',
    'dimensions_height': 'float',
    'dimensions_width': 'float',
    'dimensions_depth': 'float',
    'weight': 'float',
    'screen_size': 'float',
    'screen_resolution': 'float',
    'price_min': 'float',
    'price_max': 'float',
    'lazada_ratings': 'float',

    # int
    'camera_count_back': 'int',
    'camera_count_front': 'int',
    'benchmark_antutu': 'int',
    'benchmark_geekbench': 'int',
    'ram_min': 'int',
    'ram_max': 'int',
    'storage_min': 'int',
    'storage_max': 'int',
    'lazada_reviews': 'int',
    'release': 'int'

}

courses_df = courses_df.astype(dt)

comb_frame = courses_df.select_dtypes('string').agg(' '.join, axis=1)
comb_frame = comb_frame.replace({"[^A-Za-z0-9 ]+": ""}, regex=True)

# Add clustering labels to every non-retired course
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(comb_frame)

course_df = pd.read_csv("clean_phone_data.csv")
course_df = course_df.astype(dt)

course_df['InputString'] = course_df.select_dtypes('string').agg(' '.join, axis=1)


course_df['ClusterPrediction'] = ""


def cluster_predict(str_input):
    Y = vectorizer.transform(list(str_input))
    prediction = model.predict(Y)
    return prediction

# Cluster category for each live course
course_df['ClusterPrediction'] =  course_df.apply(lambda x: cluster_predict(course_df['InputString']), axis=0)


def recommend_util(str_input):
    
    # Predict category of input string category
    temp_df = course_df.copy()
    temp_df = temp_df.astype(dt)
    temp_df['InputString'] = temp_df.select_dtypes('string').agg(' '.join, axis=1)
    str_input = list(temp_df['InputString'])
    
    prediction_inp = cluster_predict(str_input)
    prediction_inp = int(prediction_inp)
    
    temp_df = course_df.loc[course_df['ClusterPrediction'] == prediction_inp]
    temp_df = temp_df.sample(10)
    
    return list(temp_df['CourseId'])

if __name__ == '__main__':
    queries = ['wp7-core', 'ef41-data-access', 'nosql-big-pic', 'procedural-ice-modeling-softimage-153', \
               'beginners-guide-shading-networks-softimage-510', 'centralized-logging-elastic-stack', \
               'apache-pig-data-transformations']

    for query in queries:
        res = recommend_util(query)
        print(res)

ValueError: X has 1576 features, but KMeans is expecting 595 features as input.